## Imports

In [1]:
import asyncio 
import io
import pandas as pd
import time
import numpy as np
import datetime as dt
from threading import Thread
import os
from multiprocessing import Pool

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 51)
pd.options.display.max_colwidth = 500
pd.options.display.float_format ='{:.3f}'.format

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
file_list = os.listdir('Samples3/')

## Custom Fuctions

In [4]:
def end_of_client(lt):
    n = 0
    while lt[n] != 0: n = n+1
    lt[n] = 1
    return lt

In [5]:
def data_up():
    global CLDF_LIST, OPDF, CLDF, file_list
    file_list = os.listdir('Samples3/')
    file_list.remove('.ipynb_checkpoints')
    CLDF_LIST =  []
    for filename in file_list:
        df = pd.read_csv(f'Samples3/{filename}').sort_values('Input')
        df.columns = ['id', 'start_time', 'task_type', 'wait_ready', 'psi', 'task_time']
        df['hard_level'] = np.random.randint(1, 10, df.shape[0])
        df['operator_id'] = 0
        df['day_part'] = 'lanch'
        df['end'] = 0
        df['task_type'] = 'w' + df['task_type'].astype(str)
        part1 = 30000
        part2 = 60000
        df.loc[df['start_time'] < part1, 'day_part'] = 'morning'
        df.loc[df['start_time'] > part2, 'day_part'] = 'evening'
        df['late'] = 0
        df['end_time_predict'] = 0
        CLDF_LIST.append(df)
    CLDF = CLDF_LIST[0]
    OPDF = pd.read_csv('OperatorDF.csv', sep = ';')
    OPDF['list_of_clien'] = ''
    OPDF['end_time_predict'] = 0
    OPDF['client_count'] =0
    OPDF['hard_sum'] = 0
    

In [6]:
psi_matrix = pd.read_csv('Psi_Matrix.csv', sep = ';')
Operator_time_predict = pd.read_csv('Operators_time_predict.csv', sep = ';')
Customer_time_to_wait = pd.read_csv('Customer_time_to_wait.csv', sep = ';')

In [7]:
data_up()

# Step 1

In [8]:
def alg_FIFO(num):
    global CLDF_LIST, OP, Operator_time_predict, psi_matrix, start_time 
    #обновляем данные
    data_up()
    #выбираем конкретных клиентов
    CL = CLDF_LIST[num]
    OP = OPDF.copy()
    size  = CL.shape[0]
    #запускаем счётчик времени
    for now_time in range (0, CL.head(OP.shape[0])['start_time'].max(), 1):
        #запускаем пробег по всем операторам и проверяем, есть ли среди них освободившихся
        for check in OP.id:
            if (OP.loc[OP['id'] == check, 'end_time_predict'].item() <= now_time) & ((CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)].shape[0] > 0)):
                
                #Берём первого клиента
                BestID = CL.loc[(CL['end'] == 0) & (CL['start_time'] <= now_time), 'id'].head(1).item()
                
                #меняем время освободжения первому оператору 
                OP.loc[OP['id'] == check, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'] +\
                CL.loc[CL['id'] == BestID, 'task_time'].item()

                
                #меняем время завершение клиенту
                CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()   
                
                #записываем оператору выбранного клиента
                OP.loc[OP['id'] == check, 'list_of_clien'] =  OP.loc[OP['id'] == check, 'list_of_clien'] + ' | ' + str(BestID)
                
                #помечаем, что клиента взяли
                CL.loc[CL['id'] == BestID, 'end'] = 1
                
                #записываем, какой именно оператор взял клиентв
                CL.loc[CL['id'] == BestID, 'operator_id' ] = check
                
                #записываем клиенту схожесть психотипов
                CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                              str(OP.loc[OP['id']==check, 'psi'].item())].item()
                
                #записываем выбранному оператору количество клиентов и их суммарную сложность
                OP.loc[OP['id'] == check, 'client_count'] += 1
                OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
    while CL.end.sum() != size:
        #запускаем пробег по всем операторам и проверяем, есть ли среди них освободившихся
        for check in OP.id:
            if (OP.loc[OP['id'] == check, 'end_time_predict'].item() <= now_time) & ((CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)].shape[0] > 0)):
                
                #Берём первого клиента
                BestID = CL.loc[(CL['end'] == 0) & (CL['start_time'] <= now_time), 'id'].head(1).item()
                
                #меняем время освободжения первому оператору 
                OP.loc[OP['id'] == check, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'] +\
                CL.loc[CL['id'] == BestID, 'task_time'].item()

                
                #меняем время завершение клиенту
                CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()   
                
                #записываем оператору выбранного клиента
                OP.loc[OP['id'] == check, 'list_of_clien'] =  OP.loc[OP['id'] == check, 'list_of_clien'] + ' | ' + str(BestID)
                
                #помечаем, что клиента взяли
                CL.loc[CL['id'] == BestID, 'end'] = 1
                
                #записываем, какой именно оператор взял клиентв
                CL.loc[CL['id'] == BestID, 'operator_id' ] = check
                
                #записываем клиенту схожесть психотипов
                CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                              str(OP.loc[OP['id']==check, 'psi'].item())].item()
                
                #записываем выбранному оператору количество клиентов и их суммарную сложность
                OP.loc[OP['id'] == check, 'client_count'] += 1
                OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
        now_time = max(OP.end_time_predict.min(), CL[CL['end'] == 0]['start_time'].min())
    #считаем среднее запаздывание 
    CL['deadline'] = CL['start_time'] + CL['wait_ready']
    for i in CL.id:
        CL.loc[CL['id']==i, 'late'] = max(0,  (CL.loc[CL['id'] == i, 'end_time_predict'].item() - CL.loc[CL['id'] == i, 'deadline'].item())   )
    f1 = CL[CL['end'] == 1].late.mean()
    f2 = CL[CL['end'] == 1].match.mean()
    OP['tired'] = OP['hard_sum'] / OP['client_count'] / OP['h_lvl']
    f3 = OP.tired.mean()
    out= [f1, f2, f3, file_list[num]]
    results = pd.DataFrame([out], columns=['f1', 'f2', 'f3', 'file_name'])
    print(f'''{file_list[num]} - END | {dt.datetime.now().strftime("%H:%M:%S")} \n''')
    return (results)

In [ ]:
%%time
start_time = time.time()
if __name__ == '__main__':
    with Pool(processes=20) as p:
        answer_FIFO = p.map(alg_FIFO, list(np.arange( len(file_list) )))

In [10]:
df_FIFO = pd.concat(answer_FIFO)

In [11]:
df_FIFO.to_csv('df_FIFO.csv', index  = False)

## G creating

In [17]:
def get_g1(df):
    global Operator_time_predict, FreeOp, now_time
    df['g1'] = 0
    for i in df.id:
        df.loc[df['id'] == i, 'g1'] =\
        df.loc[df['id'] == i, 'wait_ready'].item()\
        - (now_time - df.start_time) \
        - Operator_time_predict.loc[Operator_time_predict['id'] == FreeOp.head(1).id.item(), df.loc[df['id'] == i, 'task_type'].item()].item()
    df['g1'] = np.log(df['g1'].astype(int) + 10000)
    return df

In [18]:
def get_g2(df):
    global Operator_time_predict, FreeOp
    df['g2'] = 0
    df['work_time_predict'] = ''
    for i in df.id:
        df.loc[df['id'] == i, 'work_time_predict'] =\
        Operator_time_predict.loc[Operator_time_predict['id'] == FreeOp.id.item(), 
                                  df.loc[df['id'] == i, 'task_type'].item()].item()
    df['g2'] = np.log(df['work_time_predict'].astype(int))
    return df

In [19]:
def get_g3(df, FreeOp):
    global psi_matrix
    df['g3'] = 0
    for i in df.id:
        df.loc[df['id'] == i, 'g3'] = psi_matrix.loc[psi_matrix['psi'] == df.loc[df['id'] == i, 'psi'].item(),
                f'{FreeOp.psi.item()}'].item()
    return df

In [20]:
def get_KST(df):
    global Operator_time_predict, FreeOp, now_time
    df['K_stock'] = 0
    for i in df.id:
        df.loc[df['id'] == i,'K_stock'] =\
                    (df['start_time'] + df['wait_ready']  - now_time)\
                    / Operator_time_predict.loc[   Operator_time_predict['id'] == FreeOp.head(1)['id'].item(),
                                                   df.loc[df['id'] == i, 'task_type'].item()         ].item()
    return df

# Step 2

In [21]:
%%time
def alg_ABoM(num):
    global CLDF_LIST, OP, Operator_time_predict, psi_matrix
    #обновляем данные
    data_up()
    #выбираем пачку клиентов
    CL = CLDF_LIST[num]
    OP = OPDF.copy()
    size  = CL.shape[0]
    print(f"#{num}", end = '| ')
    #запускаем счетчик врмение
    for now_time in range (0, CL.head(OP.shape[0]+1)['start_time'].max(), 1):
        #запускаем пробег по всем операторам и проверяем, есть ли среди них освободившихся
        for check in OP.id:
            if (OP.loc[OP['id'] == check, 'end_time_predict'].item() <= now_time) & ((CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)].shape[0] > 0)):
    
                #выделяем всех свободных оператороов
                FreeOp = OP.loc[OP['id'] == check, :]
                #выделяем всех ожидающих клиентов
                WaitClient = CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)]
                #расчитываем подходимость по психотипу
                WaitClient = get_g3(WaitClient, FreeOp)
                
                #ищем клиента с максимальной схожестью
                BestID = WaitClient[WaitClient['g3'] == WaitClient['g3'].max()].head(1)['id'].item()
    
                #меняем время освободжения первому оператору 
                OP.loc[OP['id'] == check, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'] +\
                Operator_time_predict.loc[ Operator_time_predict['id'] == check,
                CL.loc[CL['id'] == BestID, 'task_type'].item()]
                
                #меняем время особождения клиента
                CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()         
                #записываем выбранного клиента в список к оператору
                OP.loc[OP['id'] == check, 'list_of_clien'] =  OP.loc[OP['id'] == check, 'list_of_clien'] + ' | ' + str(BestID)
                #записываем, что клиент занят
                CL.loc[CL['id'] == BestID, 'end'] = 1
                CL.loc[CL['id'] == BestID, 'operator_id' ] = check
                #записываем величину матча
                CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                              str(OP.loc[OP['id']==check, 'psi'].item())].item()
                
                  #записываем выбранному оператору количество клиентов и их суммарную сложность
                OP.loc[OP['id'] == check, 'client_count'] += 1
                OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
                #print(now_time, end ="|")
    now_time = max( OP[OP['end_time_predict'] > 0].end_time_predict.min(),  CL[CL['end'] == 0]['start_time'].min())
    start_time = time.time()
    while CL.end.sum() != size:
        #запускаем пробег по всем операторам и проверяем, есть ли среди них освободившихся
                check = OP.loc[OP['end_time_predict'] <= now_time, 'id'].head(1).item()
                #выделяем всех свободных оператороов
                FreeOp = OP.loc[OP['id'] == check, :]
                #выделяем всех ожидающих клиентов
                WaitClient = CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)]
                #расчитываем подходимость по психотипу
                WaitClient = get_g3(WaitClient, FreeOp)
                
                #ищем клиента с максимальной схожестью
                BestID = WaitClient[WaitClient['g3'] == WaitClient['g3'].max()].head(1)['id'].item()
    
                #меняем время освободжения первому оператору 
                OP.loc[OP['id'] == check, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'] +\
                Operator_time_predict.loc[ Operator_time_predict['id'] == check,
                CL.loc[CL['id'] == BestID, 'task_type'].item()]
                
                #меняем время особождения клиента
                CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()         
                #записываем выбранного клиента в список к оператору
                OP.loc[OP['id'] == check, 'list_of_clien'] =  OP.loc[OP['id'] == check, 'list_of_clien'] + ' | ' + str(BestID)
                #записываем, что клиент занят
                CL.loc[CL['id'] == BestID, 'end'] = 1
                CL.loc[CL['id'] == BestID, 'operator_id' ] = check
                #записываем величину матча
                CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                              str(OP.loc[OP['id']==check, 'psi'].item())].item()
                
                  #записываем выбранному оператору количество клиентов и их суммарную сложность
                OP.loc[OP['id'] == check, 'client_count'] += 1
                OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
                now_time = max(OP.end_time_predict.min(), CL[CL['end'] == 0]['start_time'].min())
    #считаем среднее запаздывание
    CL['deadline'] = CL['start_time'] + CL['wait_ready']
    for i in CL.id:
        CL.loc[CL['id']==i, 'late'] = max(0,  (CL.loc[CL['id'] == i, 'end_time_predict'].item() - CL.loc[CL['id'] == i, 'deadline'].item())   )
    OP['tired'] = OP['hard_sum'] / OP['client_count'] / OP['h_lvl']
    f1 = CL[CL['end'] == 1].late.mean()
    f2 = CL[CL['end'] == 1].match.mean()
    f3 = OP.tired.mean()
    out= [f1, f2, f3, file_list[num]]
    results = pd.DataFrame([out], columns=['f1', 'f2', 'f3', 'file_name'])
    return (results)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [22]:
%%time
start_time = time.time()

end = list(np.arange(640))
if __name__ == '__main__':
    with Pool(processes = 20) as p:
        anwer_AMoB = p.map(alg_ABoM,  list(np.arange( len(file_list) )) )

#1#0| | 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

CPU times: user 38.4 ms, sys: 121 ms, total: 160 ms
Wall time: 6.77 s


In [ ]:
file

In [23]:
df_ABoM = pd.concat(anwer_AMoB)

In [26]:
df_ABoM.to_csv('df_ABoM.csv', index  = False)

# Step 3

In [31]:
results = pd.DataFrame(np.array([[0, 0, 0, 0, 0, 0, 0, '']]),
                   columns=['n','L1', 'L2', 'L3', 'f1', 'f2', 'f3', 'file_name'])
now_time = 0

In [34]:
def alg_DCSF(num):
    global CLDF_LIST, OPDF, WaitClient, results, FreeOp
    print(f'''{file_list[num]} - START | {dt.datetime.now().strftime("%H:%M:%S")} \n ''')
    data_up()
    n = 0
    end = 1
    for l1 in np.arange(0, end, 0.1):
        for l2 in np.arange(0, end - l1, 0.1):
            l3 = (-(1 - l1 - l2))
            l1 = l1
            l2 = l2
            n += 1
            #print(f'{n}| l1 ={l1:.2f}, l2 = {l2:.2f}, l3 = {l3:.2f}, sum = {abs(l1) + abs(l2) + abs(l3)}', end = " ----> ")
            CL = CLDF_LIST[num].copy()
            OP = OPDF.copy()
            #запускаем счётчик по времени
            for now_time in range (CL.start_time.min(), CL.head(OP.shape[0] + 2)['start_time'].max(), 1):
                    for check in OP.id:
                        if (OP.loc[OP['id'] == check, 'end_time_predict'].item() <= now_time) &\
                            ((CL[(CL['end'] == 0)  & (CL['start_time'] <= now_time)].shape[0] > 0)):
                                    FreeOp = OP.loc[OP['id'] == check, :]
                                    WaitClient = CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)]
                                    WaitClient = get_g1(WaitClient)
                                    WaitClient = get_g2(WaitClient)
                                    WaitClient = get_g3(WaitClient, FreeOp)
                                    WaitClient = get_KST(WaitClient)
                                    WaitClient['g_sum'] = WaitClient['K_stock']*(l1*WaitClient['g1'] + 
                                                                                 l2*WaitClient['g2'] + 
                                                                                 l3*WaitClient['g3'])
                                    #Ищем клиента с самым маленьким весом
                                    BestID = WaitClient[WaitClient['g_sum'] == WaitClient['g_sum'].min()].head(1)['id'].item()
                                    #<--- нашли того клиента, которого надо назначить освободившемуся клиенту
                                    
                                    #меняем время освобождения у подходящего свободного оператора
                                        #ВОТ ТУТ НАДО БУДЕТ ДОПОЛНИТЬ, ЧТО МЫ ПРОГНОЗИРУЕМ ПО ПРИМЕРНОМУ ВРЕМЕНИ
                                        #НО ВСТАВЛЯЕМ СРАЗУ РЕАЛЬНОЕ
                                    OP.loc[OP['id'] == check, 'end_time_predict'] = \
                                    OP.loc[OP['id'] == check, 'end_time_predict'] +\
                                    CL.loc[CL['id'] == BestID, 'task_time'].item()
                                    #WaitClient[WaitClient['id'] == BestID].head(1)['work_time_predict'].item()
                                      
                                    #ставим клиенту время окончания разговора
                                    CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()                        
                                    
                                    CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                          f'''{FreeOp.head(1)['psi'].item()}'''].item()
                                    #записываем свободному оператору подходящего клиента
                                    OP.loc[OP['id'] == check, 'list_of_clien'] =\
                                    OP.loc[OP['id'] == check, 'list_of_clien'] + '|' + str(BestID)
                                
                                    #записываем выбранному оператору количество клиентов и их суммарную сложность
                                    OP.loc[OP['id'] == check, 'client_count'] += 1
                                    OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
                
                                    #ставим выбранному клиенту статус НЕ СВОБОДЕН
                                    CL.loc[CL['id'] == BestID, 'end'] = 1
                                
                                    #записываем этому клиенту ID оператора, котрые его обработал
                                    CL.loc[CL['id'] == BestID, 'operator_id' ] = check
            now_time = max( OP[OP['end_time_predict'] > 0].end_time_predict.min(),  CL[CL['end'] == 0]['start_time'].min())
            size  = CL.shape[0]
            while CL.end.sum() != size:
                                    check = OP.loc[OP['end_time_predict'] <= now_time, 'id'].head(1).item()
                                    FreeOp = OP.loc[OP['id'] == check, :]
                                    WaitClient = CL[(CL['end'] == 0) & (CL['start_time'] <= now_time)]
                                    WaitClient = get_g1(WaitClient)
                                    WaitClient = get_g2(WaitClient)
                                    WaitClient = get_g3(WaitClient, FreeOp)
                                    WaitClient = get_KST(WaitClient)
                                    WaitClient['g_sum'] = WaitClient['K_stock']*(l1*WaitClient['g1'] + 
                                                                                 l2*WaitClient['g2'] + 
                                                                                 l3*WaitClient['g3'])
                                    BestID = WaitClient[WaitClient['g_sum'] == WaitClient['g_sum'].min()].head(1)['id'].item()
                                    
                                        #ВОТ ТУТ НАДО БУДЕТ ДОПОЛНИТЬ, ЧТО МЫ ПРОГНОЗИРУЕМ ПО ПРИМЕРНОМУ ВРЕМЕНИ
                                        #НО ВСТАВЛЯЕМ СРАЗУ РЕАЛЬНОЕ
                                    OP.loc[OP['id'] == check, 'end_time_predict'] = \
                                    OP.loc[OP['id'] == check, 'end_time_predict'] +\
                                    CL.loc[CL['id'] == BestID, 'task_time'].item()
                                    #WaitClient[WaitClient['id'] == BestID].head(1)['work_time_predict'].item()
                                      
                                    #ставим клиенту время окончания разговора
                                    CL.loc[CL['id'] == BestID, 'end_time_predict'] = OP.loc[OP['id'] == check, 'end_time_predict'].item()                        
                                    
                                    CL.loc[CL['id'] == BestID, 'match'] = psi_matrix.loc[psi_matrix['psi'] == CL.loc[CL['id'] == BestID, 'psi'].item(),
                                                                          f'''{FreeOp.head(1)['psi'].item()}'''].item()
                                    #записываем свободному оператору подходящего клиента
                                    OP.loc[OP['id'] == check, 'list_of_clien'] =\
                                    OP.loc[OP['id'] == check, 'list_of_clien'] + '|' + str(BestID)
                                    #записываем выбранному оператору количество клиентов и их суммарную сложность
                                    OP.loc[OP['id'] == check, 'client_count'] += 1
                                    OP.loc[OP['id'] == check, 'hard_sum'] += CL.loc[CL['id'] == BestID, 'hard_level'].item()
                                    #ставим выбранному клиенту статус НЕ СВОБОДЕН
                                    CL.loc[CL['id'] == BestID, 'end'] = 1
                                    #записываем этому клиенту ID оператора, котрые его обработал
                                    CL.loc[CL['id'] == BestID, 'operator_id' ] = check
                                    #print(f"now_time = {now_time}", end = "|")
                                    now_time = max(OP.end_time_predict.min(), CL[CL['end'] == 0]['start_time'].min())
            CL['deadline'] = CL['start_time'] + CL['wait_ready']
            for i in CL.id:
                CL.loc[CL['id']==i, 'late'] = max(0,  (CL.loc[CL['id'] == i, 'end_time_predict'].item() \
                                                                   - CL.loc[CL['id'] == i, 'deadline'].item())  )
            f1 = CL[CL['end'] == 1].late.mean()
            f2 = CL[CL['end'] == 1].match.mean()
            OP['tired'] = OP['hard_sum'] / OP['client_count'] / OP['h_lvl']
            f3 = OP.tired.mean()
            out= [n, l1, l2, l3, f1, f2, f3, file_list[num]]
            print(f'''Last uppdate = {dt.datetime.now().strftime("%H:%M:%S")}''', end = '\r')
            #print(f'f1 ={f1:.2f}, f2 = {f2:.2f}, l3 = {f3:.2f}', end = "|")
            results = results.append( pd.DataFrame([[n, l1, l2, l3, f1, f2, f3, file_list[num]]], columns=['n','L1', 'L2', 'L3', 'f1', 'f2', 'f3', 'file_name']) )
    print(f'''{file_list[num]} - END | {dt.datetime.now().strftime("%H:%M:%S")} \n''')
    return results

In [38]:
%%time
results = pd.DataFrame(np.array([[0, 0, 0, 0, 0, 0, 0, '']]),
                   columns=['n','L1', 'L2', 'L3', 'f1', 'f2', 'f3', 'file_name'])
print(f'''Start time = {dt.datetime.now().strftime("%H:%M:%S")}''')

if __name__ == '__main__':
    with Pool(processes = 20) as p:
        answer_DCFS = p.map(alg_DCSF, list(np.arange( len(file_list) ))  )

Start time = 21:09:31
sample0_samplesf (66).csv - START | 21:09:32 
 
sample0_samplesf (83).csv - START | 21:09:32 
 
sample0_samplesf (83).csv - END | 21:18:04 

sample0_samplesf (66).csv - END | 21:18:23 

CPU times: user 729 ms, sys: 548 ms, total: 1.28 s
Wall time: 8min 51s


In [39]:
%%time
df_DCSF = pd.concat(answer_DCFS)

CPU times: user 4.09 ms, sys: 0 ns, total: 4.09 ms
Wall time: 3.23 ms


In [40]:
df_DCSF.to_csv('df_DCSF.csv')

In [41]:
p.close()